# Analysis of A05 results

## Data loading

In [1]:
from curve_analyzer.utils.data_processing import load_curves, load_trait
curves = load_curves() # load all curves
df = load_trait("a05") # load all results for trait a05
df = df.merge(curves, "left", "curve") # join curve metadata to trait results

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'dissect')


OperationFailure: Unrecognized pipeline stage name: '$unset', full error: {'ok': 0.0, 'errmsg': "Unrecognized pipeline stage name: '$unset'", 'code': 40324, 'codeName': 'Location40324'}

In [ ]:
df = df[df["bitlength"] == 128] # filter out curves with bitlength 128
# df = df[df["cofactor"] == 1] # filter out curves with cofactor 1
# df = df[df["l"] == 3] # filter out only results for parameter l == 3

## Data inspection

In [1]:
df

NameError: name 'df' is not defined

In [ ]:
df.agg(set) # display only distinct values

In [ ]:
df.describe()

## Data visualization

In [ ]:
from curve_analyzer.utils.visualization import normalized_barplot, normalized_bubbleplot

In [ ]:
for column in ("least", "full"):
    normalized_barplot(df, column, title=f"Normalized barplot of {column}")

In [ ]:
for l in (2, 3):
    normalized_bubbleplot(df[df["l"] == l], f"least", f"full", title=f"Normalized bubble plot for l={l}")

## Playground